In [1]:
%config IPCompleter.greedy = True
%autosave 60

Autosaving every 60 seconds


## 1. NLP

### Cargar archivo

In [1]:
import pandas as pd
import os
ruta_archivo = os.path.join("imdb_dataset.csv")
df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2').sample(10000, replace=False)
# df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2')
df_criticas

,Review,Label
44992,With part reconstruction and part direct shoot...,pos
38997,I absolutely LOVED this movie when I was a kid...,pos
7668,Nickelodeon has gone down the toilet. They hav...,neg
5540,"First of all, I think the casting and acting w...",neg
14,This film is absolutely appalling and awful. I...,neg
...,...,...
37827,"Ok, so it's not a masterpiece like the Godfath...",pos
16346,We have a character named Evie. Evie just want...,pos
27666,"Oh, brother...after hearing about this ridicul...",neg
19557,I wish I was first exposed to this in a movie ...,pos


### Palabras de parada

In [2]:
import nltk
import string
# nltk.download('punkt')
# nltk.download('stopwords')
palabras_de_parada_ingles = set(nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['...'])
palabras_de_parada_ingles

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '...',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself'

### Normalizar el texto

In [16]:
#pip install pycontractions

In [3]:
import re

def limpiar_html(raw_html):
    reg_limpiar = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    texto_limpio = re.sub(reg_limpiar, '', raw_html)
    return texto_limpio

def normalizar_critica(critica):
    critica_sin_html = limpiar_html(critica)
    tokens = nltk.tokenize.casual.casual_tokenize(critica_sin_html, "english")
    tokens_normalizados = [token.lower() for token in tokens if token not in palabras_de_parada_ingles]
    return " ".join(tokens_normalizados)

def normalizar_fila(fila):
    nueva_fila = fila
    critica_normalizada = normalizar_critica(fila['Review'])
    nueva_fila['Review'] = critica_normalizada
    return nueva_fila

In [4]:
df_criticas = df_criticas.apply(normalizar_fila, axis=1)
df_criticas

,Review,Label
44992,with part reconstruction part direct shooting ...,pos
38997,i absolutely loved movie i kid i cried every t...,pos
7668,nickelodeon gone toilet they kids saying thing...,neg
5540,first i think casting acting excellent the pro...,neg
14,this film absolutely appalling awful it's low ...,neg
...,...,...
37827,ok masterpiece like godfather the purpose movi...,pos
16346,we character named evie evie wants good person...,pos
27666,oh brother hearing ridiculous film umpteen yea...,neg
19557,i wish i first exposed movie theater first rel...,pos


### Obtener el vocabulario del problema

In [5]:
def obtener_vocabulario_problema(df_criticas_normalizadas):
    cuerpo_texto_normalizado = ' '.join(df_criticas_normalizadas['Review'].tolist())
    vocabulario_problema = cuerpo_texto_normalizado.split()
    vocabulario_ordenado = sorted(set(vocabulario_problema))
    return vocabulario_ordenado

def obtener_vocabulario_problema_y_posicion(vocabulario_del_problema_ordenado):
    vocabulario_y_posicion = {}
    for i, token in enumerate(vocabulario_del_problema_ordenado):
        vocabulario_y_posicion[token] = i
    return vocabulario_y_posicion

In [6]:
vocabulario_problema = obtener_vocabulario_problema_y_posicion(obtener_vocabulario_problema(df_criticas))
#vocabulario_problema.keys()

### One Hot Encoding

In [7]:
import numpy as np

def obtener_one_hot_vector(critica, vocabulario_problema_y_posicion):
    one_hot_vector = np.zeros(len(vocabulario_problema_y_posicion), dtype=int)
    for token in critica.split():
        one_hot_vector[vocabulario_problema_y_posicion[token]] = 1
    return one_hot_vector

In [8]:
one_hots = []
indices = []
for indice, fila in df_criticas.iterrows():
    one_hot = obtener_one_hot_vector(fila['Review'], vocabulario_problema)
    indices.append(indice)
    one_hots.append(one_hot)

## 2. Crear y entrenar el modelo predictivo

In [10]:
# se cambia el nombre de la variable 'one_hots' para mas claridad
X = one_hots
Y = df_criticas['Label'].ravel()

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [11]:
import gc
# Liberar memoria RAM
lst = [df_criticas]
del df_criticas, one_hots, X, Y
del lst
gc.collect()

In [ ]:
from sklearn.linear_model import LogisticRegression
clasificador_reg_log = LogisticRegression(random_state=0, solver='liblinear') #Falta añadir el solver del magister...
clasificador_reg_log.fit(X_train, Y_train)
print('accuracy del clasificador: {0:.2f}'.format(clasificador_reg_log.score(X_test, Y_test)))

### Guardar el clasificador como un archivo

In [34]:
import pickle
ruta_archivo_clasificador = os.path.join('clasificador-regresion-logistica.pkl')
archivo_clasificador = open(ruta_archivo_clasificador, 'wb')
pickle.dump(clasificador_reg_log, archivo_clasificador)
archivo_clasificador.close()

### Guardar el vocabulario del problema como un archivo

In [35]:
ruta_archivo_vocabulario = os.path.join('vocabulario-problema.pkl')
archivo_vacabulario = open(ruta_archivo_vocabulario, 'wb')
pickle.dump(vocabulario_problema, archivo_vacabulario)
archivo_vacabulario.close()

### Guardar las palabras de parada como un archivo

In [36]:
ruta_archivo_palabras_parada = os.path.join('palabras-parada.pkl')
archivo_palabras_parada = open(ruta_archivo_palabras_parada, 'wb')
pickle.dump(palabras_de_parada_ingles, archivo_palabras_parada)
archivo_palabras_parada.close()